# 혁신의 숲 크롤링

In [ ]:
#패키지 설치
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

import re
import time
import datetime
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [ ]:
#셀레니움 열기
path=r'C:\Users\NTX550\Desktop\chromedriver-win64\chromedriver.exe'

s=Service(path)
driver=webdriver.Chrome(service=s)
driver.set_window_size(700,700)

In [ ]:
#저장할 데이터와 코드를 읽을 수 있는 파일 열기
df=pd.read_excel('Company_code.xlsx')
df.drop(['Unnamed: 0'],axis=1,inplace=True)
df2=pd.read_excel('THOUSAND_7.xlsx')
df2.drop(['Unnamed: 0'],axis=1,inplace=True)
df2.head()

In [ ]:
#크롤링
for idx in range(len(df)):

    if idx % 200 == 0:
        df2.to_excel(f"temp{idx}.xlsx")
    print(idx)
    print(df.at[idx,'code'])
    code=df.at[idx,'code']
    url=f'https://www.innoforest.co.kr/company/{code}'
    driver.get(url)
    time.sleep(3)

    soup=BeautifulSoup(driver.page_source)


    #기업정보
    info=soup.find('div',class_='css-4q8h6z')
    a=info.find_all('div',class_='css-198bnr5')
    c=['listed','operation','make_date','homepage','address']
    for i in range(len(c)):
        if len(c)!=len(a):
            error.append(code)
        else:
            df2.at[idx,c[i]]=a[i].find('dd').text
            print(a[i].find('dd').text)


    #임직원
    df2.at[idx,'name']=''
    df2.at[idx,'link']=''

    cla=['css-1ya8xf7']
    for i in range(len(cla)):
        a=soup.find('div',class_=cla[i]).text
        df2.at[idx,'name']=a.split('*')[0]
        b=''
        if soup.find('div',class_=cla[i]).find('a'):
            b=soup.find('div',class_=cla[i]).find('a').get('href')
            df2.at[idx,'link']=b



    #주요정보
    main=soup.find('dl',class_='css-1lhycl7').find_all('dd',class_='css-ol6q4q')
    #자본금,고용인원,누적투자유치금액,투자유치횟수,연매출,기술등급
    c2=['seed','employer','acc_invest','cnt_invest','year_sales','tech_rank']
    for i in range(len(main)):
        #자본금
        fin=main[i].text
        df2.at[idx,c2[i]]=main[i].text


    #기업설명
    df2.at[idx,'Company_explain']=info.find('p').text

    #검은색해시태그
    df2.at[idx,'Main_keyword']=''
    black=info.find_all('span',class_='css-1t6caqs')
    blackhash=[]
    for i in range(len(black)):
        a=info.find_all('span',class_='css-1t6caqs')[i].text
        for j in range(len(a)):
            if a[j].isdigit():
                if j==len(a):
                    break
                elif a[j+1].isdigit():
                    break
        blackhash.append(a[:j])
    print(blackhash)
    df2.at[idx,'Main_keyword']=blackhash

    #하얀색 해시태그
    df2.at[idx,'other_keyword']=''
    grey=info.find_all('span',class_='css-184w0e7')
    greyhash=[]
    for i in range(len(grey)):
        a=info.find_all('span',class_='css-184w0e7')[i].text
        for j in range(len(a)):
            if a[j].isdigit():
                if j==len(a)-1:
                    break
                elif a[j+1].isdigit():
                    break
        greyhash.append(a[:j])
    df2.at[idx,'other_keyword']=greyhash
    print(greyhash)
    print('*'*50)

In [ ]:
#파일저장
df2.to_excel('Info_company.xlsx')

# 빅카인즈 크롤링

In [ ]:
#패키지 설치
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

import re
import time
import datetime
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [ ]:
df=pd.read_excel('temp1.xlsx')
df.drop(['Unnamed: 0'],axis=1,inplace=True)

In [ ]:
#셀레니움 열기
path=r'C:\Users\NTX550\Desktop\chromedriver-win64\chromedriver.exe'

s=Service(path)
driver=webdriver.Chrome(service=s)
driver.set_window_size(700,700)

In [ ]:
for i in range(5000,5500):
    #페이지 열기
    url='https://www.bigkinds.or.kr/v2/news/index.do'
    driver.get(url)
    time.sleep(1)

    #검색어
    query_txt=df.at[i,'Company']

    #검색어 입력할 곳
    element=driver.find_element(By.XPATH,'//*[@id="total-search-key"]')

    #검색어 입력
    element.send_keys(query_txt)

    #검색어 찾기
    element.send_keys(Keys.ENTER)
    #btn_path='//*[@id="search-foot-div"]/div[2]/button[2]'
    #driver.find_element(By.XPATH,btn_path).click()

    time.sleep(2)
    #건수
    soup=BeautifulSoup(driver.page_source)
    df.at[i,'news']=int(soup.find('span',class_='total-news-cnt').text.replace(',',''))
    #p='//*[@id="news-results-tab"]/div[3]/h3/span[6]'
    #df.at[i,'news']=driver.find_element(By.XPATH,p).text
    print(i,(soup.find('span',class_='total-news-cnt').text).replace(',',''))

In [ ]:
df.to_excel('temp.xlsx')